In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install torchao==0.14.0 fbgemm-gpu-genai==1.3.0
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [2]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


TMA benchmarks will be running without grid constant TMA descriptor.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length = 2048,
    load_in_4bit = False,
    load_in_8bit = False,
    full_finetuning = False,
)

==((====))==  Unsloth 2025.10.12: Fast Qwen3 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    qat_scheme = "int4",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Applying QAT to mitigate quantization degradation


Unsloth 2025.10.12 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [5]:
for module in model.modules():
    if "FakeQuantized" in module.__class__.__name__:
        print("QAT is applied!")
        break

QAT is applied!


In [6]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-instruct",
)

In [7]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [9]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [10]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [11]:
dataset[10]['text']

"<|im_start|>user\nWrite Python code to solve the task:\nIn programming, hexadecimal notation is often used.\n\nIn hexadecimal notation, besides the ten digits 0, 1, ..., 9, the six letters `A`, `B`, `C`, `D`, `E` and `F` are used to represent the values 10, 11, 12, 13, 14 and 15, respectively.\n\nIn this problem, you are given two letters X and Y. Each X and Y is `A`, `B`, `C`, `D`, `E` or `F`.\n\nWhen X and Y are seen as hexadecimal numbers, which is larger?\n\nConstraints\n\n* Each X and Y is `A`, `B`, `C`, `D`, `E` or `F`.\n\nInput\n\nInput is given from Standard Input in the following format:\n\n\nX Y\n\n\nOutput\n\nIf X is smaller, print `<`; if Y is smaller, print `>`; if they are equal, print `=`.\n\nExamples\n\nInput\n\nA B\n\n\nOutput\n\n<\n\n\nInput\n\nE C\n\n\nOutput\n\n>\n\n\nInput\n\nF F\n\n\nOutput\n\n=<|im_end|>\n<|im_start|>assistant\nStep 1:  To compare two hexadecimal numbers represented by the letters X and Y, we need to understand the hexadecimal number system and 

In [12]:
from trl import SFTTrainer, SFTConfig

In [13]:
args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
        output_dir = "outputs",
    )

In [14]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = args
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [15]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Step,Training Loss
1,0.992800
2,3.271300
3,1.123400
4,1.382500
5,2.229300
6,0.947000
7,0.447500
8,0.848900
9,0.548300
10,0.568400


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
# Save the LoRA adapters
model.save_pretrained("qwen3_4b_lora_model")
tokenizer.save_pretrained("qwen3_4b_lora_model")

# Or save in different formats:
# Save as 16-bit merged model
model.save_pretrained_merged("qwen3_4b_merged_16bit", tokenizer, save_method = "merged_16bit")

# Save as 4-bit quantized model
model.save_pretrained_merged("qwen3_4b_merged_4bit", tokenizer, save_method = "merged_4bit")

# Save for GGUF (for llama.cpp usage)
model.save_pretrained_gguf("qwen3_4b_gguf", tokenizer, quantization_method = "q4_k_m")

In [ ]:
FastLanguageModel.for_inference(model)

In [19]:
messages = [
    {"role": "user", "content": "What is machine learning?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt"
).to("cuda")

In [20]:
outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 128,
    temperature = 0.7,
    top_p = 0.9,
    use_cache = True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


user
What is machine learning?
assistant
Machine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn patterns and make predictions or decisions without being explicitly programmed for the task. It uses large datasets to identify patterns and relationships between variables, enabling machines to make predictions or decisions based on new data. Machine learning has numerous applications, including image recognition, natural language processing, and recommendation systems.


In [21]:
test_examples = [
    "Explain quantum computing",
    "Write a Python function to reverse a string",
    "What are the benefits of exercise?"
]

for example in test_examples:
    messages = [{"role": "user", "content": example}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt"
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 128)
    print(f"\nQ: {example}")
    print(f"A: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")


Q: Explain quantum computing
A: user
Explain quantum computing
assistant
Quantum computing is a rapidly advancing field of computer science that leverages the principles of quantum mechanics to perform calculations and solve problems that are beyond the capabilities of classical computers. Unlike classical computers that use bits as the basic unit of information, quantum computers use quantum bits or qubits as their fundamental building blocks.

Qubits are unique in that they can exist in a superposition of both 0 and 1 simultaneously, allowing quantum computers to perform multiple calculations at once. This property enables quantum computers to explore a vast number of possible solutions to complex problems much more efficiently than classical computers.

Furthermore, quantum computers can also exploit entanglement, a

Q: Write a Python function to reverse a string
A: user
Write a Python function to reverse a string
assistant
Here's a Python function to reverse a string:

```python
d